In [1]:
from os import listdir, walk
from os.path import isfile, join
import numpy as np
from PIL import Image
import cv2
from IPython.display import display
import math, random
import time, datetime, sys, os, shutil
import operator

import tensorflow as tf
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Input
from tensorflow.keras.utils import Sequence
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.optimizers import *

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split
import pandas as pd

print(tf.config.experimental.list_physical_devices())

BATCH_SIZE = 32

%run ./variables.ipynb
%run ./utils.ipynb
%run ../utils/data_utils.ipynb

id_map = get_selected_taxons(SELECTED_TAXONS)
id_map_inv = get_selected_taxons(SELECTED_TAXONS, inv=True)
n_classes = len(list(id_map.keys()))
check_dirs(SAVED_MODELS_ROOT)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'), PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# Handle data

## Preparing panda arrays

In [2]:
X_train, y_train, _ = get_dataset(TRAIN_DATASET_PATH, ids=False)
X_test, y_test, _ = get_dataset(TEST_DATASET_PATH, ids=False)

print("Test length:", len(X_train), "- n classes:", len(np.unique(y_train)))
print("Test length:", len(X_test), "- n classes:", len(np.unique(y_test)))

# Balance dataset
#X_train, y_train, max_samples = balance_dataset(X_train, y_train, max_samples=None)

Retrieving dataset from: /mnt/nvme-storage/pfauregi/training/thumbails/ADIAC_D38/dataset/train/


'38/38'

Retrieving dataset from: /mnt/nvme-storage/pfauregi/training/thumbails/ADIAC_D38/dataset/test/


'38/38'

Test length: 753 - n classes: 38
Test length: 84 - n classes: 38


In [3]:
data_train = {'png_path':  X_train, 'taxon_id': y_train}
data_test = {'png_path':  X_test, 'taxon_id': y_test}

df_train = pd.DataFrame(data_train, columns = ['png_path', 'taxon_id'])
df_test = pd.DataFrame(data_test, columns = ['png_path', 'taxon_id'])
df_train = df_train.sample(frac=1).reset_index(drop=True)
df_test = df_test.sample(frac=1).reset_index(drop=True)

# Prtining some infos
print(len(df_train), len(df_test))
df_train.head()
df_test.head()

753 84


,png_path,taxon_id
0,/mnt/nvme-storage/pfauregi/training/thumbails/...,Tabularia_investiens
1,/mnt/nvme-storage/pfauregi/training/thumbails/...,Cocconeis_stauroneiformis
2,/mnt/nvme-storage/pfauregi/training/thumbails/...,Opephora_olsenii
3,/mnt/nvme-storage/pfauregi/training/thumbails/...,Tabellaria_quadriseptata
4,/mnt/nvme-storage/pfauregi/training/thumbails/...,Gomphonema_sp.1


## Setting up the flows

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255.,
                         rotation_range=20, 
                         brightness_range=[0.8,1.2], 
                         horizontal_flip=True, 
                         vertical_flip=True,
                         fill_mode='nearest',
                         width_shift_range=10,
                         height_shift_range=10,
                         zoom_range=0.2,
                         data_format="channels_last")

test_datagen = ImageDataGenerator(rescale=1./255.,
                         data_format="channels_last")

In [5]:
#classes_array = np.unique(y_train)
train_generator = train_datagen.flow_from_dataframe(
        dataframe=df_train,
        x_col='png_path',
        y_col='taxon_id',
        target_size=(256, 256),
        batch_size=32,
        #classes=classes_array,
        class_mode='categorical')
val_generator = test_datagen.flow_from_dataframe(
        dataframe=df_test,
        x_col='png_path',
        y_col='taxon_id',
        target_size=(256, 256),
        batch_size=32,
        classes=train_generator.class_indices,
        class_mode='categorical')

train_spe = train_generator.samples // BATCH_SIZE
val_spe = val_generator.samples // BATCH_SIZE

Found 753 validated image filenames belonging to 38 classes.
Found 84 validated image filenames belonging to 38 classes.


In [6]:
class_indices = train_generator.class_indices
f = open(os.path.join(SAVED_MODELS_ROOT, 'model_id_map.csv'), 'w')
with f:
    writer = csv.writer(f)
    writer.writerow(["taxon", "id"])
    for taxon in class_indices:
        writer.writerow([taxon, class_indices[taxon]])

## Testing

In [7]:
'''
i = 0
stop = False
for batch in train_generator:
    images = batch[0]
    labels = batch[1]
    for i in range(images.shape[0]):
        print(np.argmax(labels[i]))
        image = (images[i,:,:,:]*255).astype('uint8')
        #print(image)
        display(Image.fromarray(image))
        i+=1
        if i>=1:
            stop = True
            break
    if stop: pass;
    #display()
'''

"\ni = 0\nstop = False\nfor batch in train_generator:\n    images = batch[0]\n    labels = batch[1]\n    for i in range(images.shape[0]):\n        print(np.argmax(labels[i]))\n        image = (images[i,:,:,:]*255).astype('uint8')\n        #print(image)\n        display(Image.fromarray(image))\n        i+=1\n        if i>=1:\n            stop = True\n            break\n    if stop: pass;\n    #display()\n"

# Model desgin

In [8]:
# fetching base model
input_tensor = Input(shape=(256, 256, 3))
#base_model = InceptionV3(weights='imagenet', input_tensor=input_tensor, include_top=False)
base_model = Xception(include_top=False, weights='imagenet', input_tensor=input_tensor, pooling=None)

In [9]:
# setting model for specifiv case
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
out = Dense(len(train_generator.class_indices.keys()), activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=out)

# Training

## Setting callbacks

In [10]:
# Setting tensorboard
check_dirs(LOG_DIR)
delete_all_files_in_folder(LOG_DIR)
log_dir = LOG_DIR + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

Deleting all files in /mnt/nvme-storage/pfauregi/training/thumbails/ADIAC_D38/tensorboard/


In [11]:
log_file=os.path.join(SAVED_MODELS_ROOT, "model.log")
#os.remove(log_file)
csv_logger = CSVLogger(log_file, append=True)

## Setting optimizer

In [12]:
optimizer = SGD(lr=0.1, decay=0.0001, momentum=0, nesterov=False)
#optimizer = "nadam"

## New layers only

In [13]:
#n_epochs = 5
n_epochs = 5
last_epoch = get_last_epoch(log_file)

print(n_epochs, "epochs composed of", train_spe, "batches (steps) of", BATCH_SIZE, "images.")

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_generator, 
          epochs=last_epoch+n_epochs, 
          steps_per_epoch=train_spe,
          use_multiprocessing=False, 
          validation_data=val_generator,
          validation_steps=val_spe,
          callbacks=[tensorboard_callback, csv_logger],
          initial_epoch=last_epoch)

5 epochs composed of 23 batches (steps) of 32 images.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 23 steps, validate for 2 steps
Epoch 242/246
23/23 [==============================] - 13s 559ms/step - loss: 3.4787 - accuracy: 0.1664 - val_loss: 3.4807 - val_accuracy: 0.1875
Epoch 243/246
23/23 [==============================] - 11s 457ms/step - loss: 2.9480 - accuracy: 0.4341 - val_loss: 3.2257 - val_accuracy: 0.2500
Epoch 244/246
23/23 [==============================] - 11s 458ms/step - loss: 2.3240 - accuracy: 0.5992 - val_loss: 3.0933 - val_accuracy: 0.2188
Epoch 245/246
23/23 [==============================] - 10s 451ms/step - loss: 1.8362 - accuracy: 0.6755 - val_loss: 2.9369 - val_accuracy: 0.1875
Epoch 246/246
23/23 [==============================] - 10s 450ms/step - loss: 1.3957 - accuracy: 0.7642 - val_loss: 2.8349 - val_accuracy: 0.2500


In [14]:
np.max(history.history["val_accuracy"])

0.25

## Training full model

In [15]:
#n_epochs = 50
n_epochs = 100
last_epoch = get_last_epoch(log_file)

print(n_epochs, "epochs composed of", train_spe, "batches (steps) of", BATCH_SIZE, "images.")

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers:
    layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
history = model.fit(train_generator, 
          epochs=last_epoch+n_epochs, 
          steps_per_epoch=train_spe,
          use_multiprocessing=False, 
          validation_data=val_generator,
          validation_steps=val_spe,
          callbacks=[tensorboard_callback, csv_logger],
          initial_epoch=last_epoch)
print(np.max(history.history["val_accuracy"]))

100 epochs composed of 23 batches (steps) of 32 images.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 23 steps, validate for 2 steps
Epoch 246/345
23/23 [==============================] - 13s 571ms/step - loss: 0.6452 - accuracy: 0.8350 - val_loss: 2.8958 - val_accuracy: 0.2500
Epoch 247/345
23/23 [==============================] - 11s 468ms/step - loss: 0.2454 - accuracy: 0.9334 - val_loss: 1.8269 - val_accuracy: 0.4844
Epoch 248/345
23/23 [==============================] - 11s 467ms/step - loss: 0.1185 - accuracy: 0.9723 - val_loss: 1.3694 - val_accuracy: 0.5781
Epoch 249/345
23/23 [==============================] - 11s 465ms/step - loss: 0.0675 - accuracy: 0.9861 - val_loss: 0.9419 - val_accuracy: 0.7344
Epoch 250/345
23/23 [==============================] - 11s 468ms/step - loss: 0.0431 - accuracy: 0.9917 - val_loss: 0.5678 - val_accuracy: 0.8125
Epoch 251/345
23/23 [==============================] - 11s 468ms/step - loss: 0.0472 - accuracy: 0.9875 - val_loss: 0.3491 

## Saving model

In [ ]:
save_model(model, SAVED_MODELS_ROOT)